In [1]:
import logging, sys
logging.disable(sys.maxsize)
import warnings
warnings.filterwarnings('ignore')

from types import SimpleNamespace
from extraction import *
import scipy.sparse as sp
from aind_ophys_utils.summary_images import pnr_image
from multiprocessing.pool import Pool

2025-01-25 00:18:49.010042: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
args = SimpleNamespace(
        input_dir="../data/",
        output_dir="../results/",
        tmp_dir="/scratch",
        diameter=0,
        anatomical_only=2,
        denoise=False,
        cellprob_threshold=0.0,
        flow_threshold=1.5,
        spatial_hp_cp=0,
        pretrained_model="cyto",
        neuropil="CNMF",
        rf=100, stride=20, K=20, merge_thr=.6, ssub=2, tsub=2, normalize_init=True, init='greedy_roi',
        contour_video=False,
    )

In [3]:
    output_dir = Path(args.output_dir).resolve()
    input_dir = Path(args.input_dir).resolve()
    tmp_dir = Path(args.tmp_dir).resolve()
    session, data_description, subject = get_metdata(input_dir)
    subject_id = subject.get("subject_id", "")
    name = data_description.get("name", "")
    setup_logging("aind-ophys-extraction-suite2p", mouse_id=subject_id, session_name=name)
    if next(input_dir.rglob("*decrosstalk.h5"), ""):
        input_fn = next(input_dir.rglob("*decrosstalk.h5"))
    else:
        input_fn = next(input_dir.rglob("*registered.h5"))
    parent_directory = input_fn.parent
    if session is not None and "Bergamo" in session["rig_id"]:
        motion_corrected_fn = bergamo_segmentation(input_fn, session, temp_dir=tmp_dir)
    else:
        motion_corrected_fn = input_fn
    if not data_description or "multiplane" in data_description.get("name", ""):
        unique_id = motion_corrected_fn.parent.parent.name
    else:
        unique_id = "_".join(str(data_description["name"]).split("_")[-3:])

    frame_rate = get_frame_rate(session)

    output_dir = make_output_directory(output_dir, unique_id)

In [4]:
big_fn = "/data/single-plane-ophys-harvard_2024-10-21_10_00_31_processed_2024-12-11_05-52-00/motion_correction/pophys_registered.h5"

In [5]:
%%time
os.environ['CAIMAN_TEMP'] = args.tmp_dir
# create mmap file
with h5py.File(big_fn) as fin:
    data = fin["data"]
    if data.nbytes < 1e9:
        fname_new = caiman.save_memmap([big_fn], var_name_hdf5='data',
                                       order='C', base_name=unique_id)
    else:
        chunksize = 50
        chunkfiles = [args.tmp_dir + f"/chunk{k}.h5" for k in range(0, data.shape[0], chunksize)]
        def write_chunk(k):
            with h5py.File(args.tmp_dir + f"/chunk{k}.h5", "w") as f:
                f.create_dataset('data', data=data[k:k+chunksize])
        with Pool() as pool:  # restricting to CO_CPUs does not speed things up
            pool.map(write_chunk, range(0, data.shape[0],chunksize))
            fname_new = caiman.save_memmap(chunkfiles, var_name_hdf5='data',
                                       order='C', dview=pool, base_name=unique_id, n_chunks=16)
            pool.map(os.remove, chunkfiles)  # remove temporary files
        # # remove temporary files
        # for cf in chunkfiles:
        #     os.remove(cf)

CPU times: user 114 ms, sys: 1.05 s, total: 1.16 s
Wall time: 39.5 s


In [6]:
%%time
os.environ['CAIMAN_TEMP'] = args.tmp_dir
# create mmap file
with h5py.File(big_fn) as fin:
    data = fin["data"]
    if data.nbytes < 1e9:
        fname_new = caiman.save_memmap([big_fn], var_name_hdf5='data',
                                       order='C', base_name=unique_id)
    else:
        chunksize = 50
        chunkfiles = [args.tmp_dir + f"/chunk{k}.h5" for k in range(0, data.shape[0], chunksize)]
        def write_chunk(k):
            with h5py.File(args.tmp_dir + f"/chunk{k}.h5", "w") as f:
                f.create_dataset('data', data=data[k:k+chunksize])
        with Pool(4) as pool:  # restricting to CO_CPUs does not speed things up
            pool.map(write_chunk, range(0, data.shape[0],chunksize))
            chunkfiles = [args.tmp_dir + f"/chunk{k}.h5" for k in range(0, data.shape[0],chunksize)]
            fname_new = caiman.save_memmap(chunkfiles, var_name_hdf5='data',
                                       order='C', dview=pool, base_name=unique_id, n_chunks=16)
            pool.map(os.remove, chunkfiles)  # remove temporary files

CPU times: user 27.9 ms, sys: 916 ms, total: 944 ms
Wall time: 44.3 s
